<a href="https://colab.research.google.com/github/diogomdbarros/automatiza-o-script/blob/main/script_usuarios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Script de Automação de Texto para Criação de Usuários


## 1. Bibliotecas

### 1.1 Importando Bibliotecas

In [ ]:
import pandas as pd

### 2. Carregando arquivo CSV

In [ ]:
df = pd.read_csv("SOLICITAÇÃO DE ACESSOS.csv")

## 3. Tratamento dos Dados

### 3.1 Removendo Colunas

Remoção de colunas que não serão utilizadas no dataframe para solução do projeto.

In [ ]:
# Exclui colunas não utilizadas do DataFrame
df.drop('MATRICULA', axis=1, inplace=True)
df.drop('DATA DE ADMISSÃO', axis=1, inplace=True)
df.drop('NOME DO SOLICITANTE', axis=1, inplace=True)
df.drop('HORARIO DE EXPEDIENTE(OPERADOR)', axis=1, inplace=True)
df.drop('Unnamed: 10', axis=1, inplace=True)
# Renomeando Colunas
df.columns = ['data', 'nome', 'setor', 'cpf', 'filial', 'telefone', 'email_solicitante', 'supervisor']

In [ ]:
df.head()

### 3.2 Função seletora de usuário espelho

Função criada para adicionar uma coluna ( setor ) ao dataframe, com dados de um usuário já existente ( usuário  espelho ) para auxílio do TI na criação dos novos acessos com as permissõs já definidas.

In [ ]:
# Função para localizar o setor e atribuir o Usuario_Espelho correspondente
def definir_usuario_espelho(setor):
    if setor == 'ADM':
        return 'joao.pedro user/0001'
    elif setor == 'RH':
        return 'maria.flor user/0002'
    elif setor == 'OPERAÇÃO':
        return 'ana.clara user/0003'
    else:
        return 'pedro.silva user/0004'

# Adiciona a coluna usuário espelho conforme o setor localizado na coluna "setor"
df['usuario_espelho'] = df['setor'].apply(definir_usuario_espelho)


### 3.3 Função Formatação de CPF

A função formata a coluna CPF verificando se os mesmos possuem 11 digitos e se todos os caracteres são numéricos.

In [ ]:
def formatar_cpf(cpf):
    # Remove qualquer caractere não numérico
    cpf = ''.join(filter(str.isdigit, cpf))
    # Garantir que o CPF tenha 11 dígitos e aplicar a formatação
    if len(cpf) == 11:
        return f'{cpf[:3]}.{cpf[3:6]}.{cpf[6:9]}-{cpf[9:]}'
    return cpf  # Retorna sem formatação se não tiver 11 dígitos

# Adiciona a formatação a coluna "cpf"
df['cpf'] = df['cpf'].apply(formatar_cpf)

### 3.4 Função Formatação de Telefone

A função formata a coluna ( telefone ) verificando se os mesmos possuem 11 digitos e se todos os caracteres são numéricos.

In [ ]:
def formatar_telefone(telefone):
    # Remove qualquer caractere não númerico
    telefone = ''.join(filter(str.isdigit, telefone))
    # Formatar telefone
    if len(telefone) == 11:  # Celular
        return f'({telefone[:2]}) {telefone[2:7]}-{telefone[7:]}'
    return telefone  # Retorna sem formatação se não tiver 11 dígitos

# Adiciona a formatação a coluna "telefone"
df['telefone'] = df['telefone'].apply(formatar_telefone)

In [ ]:
df.tail(4)

## 4. Texto Base

Aqui a variável "script" guarda o texto a ser gerado segundo as necessidades do processo. Foram adicionados placeholders {} para o preenchimento automático com os dados do dataframe.

In [ ]:
script = """
Criar AD - {nome}

Boa dia!
Poderiam realizar criação do AD {nome}, por favor?
Cargo: Teleoperador
Gerente: {gerente}
Desde já agradeço

Criar usuário - {nome}

Boa dia!
Poderiam realizar criação no SIS/ADM/QUEUE para {nome}, por favor?
Setor: {setor}
Cargo: Teleoperador
CPF: {cpf}
Gerente: {gerente}
Usuario espelho: {usuario_espelho}
Local de trabalho: {filial}
Desde já agradeço

Telefone: {telefone}

"""

# 5. Gerando Texto por Intervalo

## 5.1. O objetivo aqui é selecionar um intervalo de linhas do DataFrame df.



In [ ]:
# Selecionar linhas
df_selecionado = df.loc[df.index[466:470]]

# Gerar textos apenas para essas linhas
for index, row in df_selecionado.iterrows():
    texto_personalizado = script.format(
        nome=row['nome'],
        cpf=row['cpf'],
        setor=row['setor'],
        filial=row['filial'],
        usuario_espelho=row['usuario_espelho'],
        telefone=row['telefone']
    )
    print(f"Texto gerado para {row['nome']}:\n{texto_personalizado}\n")

## 5.2 Gerar texto pegando por Índice Expecífico



In [ ]:
# Índices das linhas que deseja processar
indices_especificos = [466, 467, 468, 469]  # Exemplo: processar a primeira, terceira e quinta linha

# Selecionar essas linhas
df_selecionado = df.loc[indices_especificos]

# Gerar textos apenas para essas linhas
for index, row in df_selecionado.iterrows():
    texto_personalizado = script.format(
         nome=row['nome'],
        cpf=row['cpf'],
        setor=row['setor'],
        filial=row['filial'],
        usuario_espelho=row['usuario_espelho'],
        telefone=row['telefone']
    )
    print(f"Texto gerado para {row['nome']}:\n{texto_personalizado}\n")
